In [49]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress
from pprint import pprint
from datetime import datetime, timedelta, timezone
import pytz
import scipy.stats
from dateutil.relativedelta import relativedelta

import warnings

# Suppress the specific warnings
warnings.filterwarnings("ignore")

today = datetime.utcnow()

from api_keys import espn_link

In [50]:
homeOnlySked = pd.read_csv('data/sked.csv', index_col=False)
homeOnlySked

completeSked = pd.read_csv('data/sked_full.csv', index_col=False)
completeSked

team_names = pd.read_csv('data/team_names.csv', index_col=False)
team_names

file_name = f"data/allG_df_fp-{today.strftime('%Y-%m-%d')}.csv"
allG_df = pd.read_csv(file_name, index_col=False)
allG_df

file_name = f"data/all_df_fp-{today.strftime('%Y-%m-%d')}.csv"
all_df = pd.read_csv(file_name, index_col=False)
all_df

team_names_only = team_names[['abbreviation', 'name']]

all_df_forwards =  all_df.loc[all_df['position'] != 'D']
all_df_defense =  all_df.loc[all_df['position'] == 'D']

all_df_centers =  all_df.loc[all_df['secondaryPosition'] == 'C']
all_df_wingers =  all_df.loc[all_df['secondaryPosition'] == 'W']

file_name = f"data/summary_stats-{today.strftime('%Y-%m-%d')}.csv"
player_bios = pd.read_csv(file_name, index_col=False)

# file_name = f"data/rankings_worksheet_{(today-timedelta(days=6)).strftime('%Y-%m-%d')}.xlsx"
# rankings = pd.read_excel(file_name)

In [51]:
startDate = '2023-12-04'
endDate = '2023-12-10'

In [52]:
lookahead_sked = homeOnlySked.loc[(homeOnlySked['gameDate'] >= startDate) & (homeOnlySked['gameDate'] <= endDate)]
lookahead_sked.sort_values('gameTime')

,gameID,gameDate,gameTime,awayTeam,homeTeam,gameDT,winningGoalie
1122,2023020405,2023-12-08,Friday 07:00 PM,STL,CBJ,2023-12-09 00:00:00,0
299,2023020406,2023-12-08,Friday 07:00 PM,PIT,FLA,2023-12-09 00:00:00,0
1159,2023020407,2023-12-08,Friday 09:00 PM,MIN,EDM,2023-12-09 02:00:00,0
709,2023020375,2023-12-04,Monday 07:00 PM,DAL,TBL,2023-12-05 00:00:00,0
627,2023020374,2023-12-04,Monday 07:00 PM,PIT,PHI,2023-12-05 00:00:00,0
464,2023020377,2023-12-04,Monday 07:30 PM,CAR,WPG,2023-12-05 00:30:00,0
915,2023020376,2023-12-04,Monday 07:30 PM,SEA,MTL,2023-12-05 00:30:00,0
544,2023020378,2023-12-04,Monday 09:00 PM,WSH,ARI,2023-12-05 02:00:00,0
94,2023020379,2023-12-04,Monday 10:00 PM,STL,VGK,2023-12-05 03:00:00,0
53,2023020408,2023-12-09,Saturday 01:00 PM,ARI,BOS,2023-12-09 18:00:00,0


In [53]:
teams_data = {}

for index, row in lookahead_sked.iterrows():

    home_team = row['homeTeam']
    away_team = row['awayTeam']

    if home_team not in teams_data:
        teams_data[home_team] = {'played': 0, 'home': 0, 'away': 0}
    if away_team not in teams_data:
        teams_data[away_team] = {'played': 0, 'home': 0, 'away': 0}

    teams_data[home_team]['played'] += 1
    teams_data[away_team]['played'] += 1
    teams_data[home_team]['home'] += 1
    teams_data[away_team]['away'] += 1

# Create a DataFrame
teams_df = pd.DataFrame.from_dict(teams_data, orient='index')
teams_df = teams_df.sort_values(by=['played', 'home'], ascending=False)

for index, row in teams_df.iterrows():
    abbrev = index
    full_name = team_names.loc[team_names['abbreviation'] == abbrev]['name'].iloc[0]
    teams_df.at[index, 'team_name'] = full_name
    
teams_df

,played,home,away,team_name
CHI,4,4,0,Chicago Blackhawks
CBJ,4,3,1,Columbus Blue Jackets
MTL,4,3,1,Montréal Canadiens
SEA,4,3,1,Seattle Kraken
VGK,4,2,2,Vegas Golden Knights
WSH,4,2,2,Washington Capitals
TBL,4,2,2,Tampa Bay Lightning
STL,4,1,3,St. Louis Blues
DAL,4,1,3,Dallas Stars
NSH,4,1,3,Nashville Predators


In [54]:
team_check = teams_df['team_name'].to_list()

nhl_teams = [
    'New Jersey Devils', 'New York Islanders', 'New York Rangers',
    'Philadelphia Flyers', 'Pittsburgh Penguins', 'Boston Bruins',
    'Buffalo Sabres', 'Montréal Canadiens', 'Ottawa Senators',
    'Toronto Maple Leafs', 'Carolina Hurricanes', 'Florida Panthers',
    'Tampa Bay Lightning', 'Washington Capitals', 'Chicago Blackhawks',
    'Detroit Red Wings', 'Nashville Predators', 'St. Louis Blues',
    'Calgary Flames', 'Colorado Avalanche', 'Edmonton Oilers',
    'Vancouver Canucks', 'Anaheim Ducks', 'Dallas Stars',
    'Los Angeles Kings', 'San Jose Sharks', 'Columbus Blue Jackets',
    'Minnesota Wild', 'Winnipeg Jets', 'Arizona Coyotes',
    'Vegas Golden Knights', 'Seattle Kraken'
]

# # teams_df.index.values

for team in nhl_teams:

    if team not in team_check:
        print(team)

In [55]:
all_df_even = all_df.copy()
all_df_even['evenGoals'] = all_df_even['goals'] - all_df_even['powerPlayGoals'] - all_df_even['shorthandedGoals']
all_df_even['powerPlayAssists'] = all_df_even['powerPlayPoints'] - all_df_even['powerPlayGoals']
all_df_even['shorthandedAssists'] = all_df_even['shPoints'] - all_df_even['shorthandedGoals']
all_df_even['evenAssists'] = all_df_even['assists'] - all_df_even['powerPlayAssists'] - all_df_even['shorthandedAssists']
all_df_even['evenPoints'] = all_df_even['assists'] + all_df_even['goals']
all_df_even['evenToi'] = all_df_even['toi'] - all_df_even['powerPlayToi'] - all_df_even['shorthandedToi']

all_df_even

,playerId,sweaterNumber,name,position,goals,assists,points,plusMinus,pim,hits,...,gameId,gamesPlayed,fantasyPoints,specialTeams,evenGoals,powerPlayAssists,shorthandedAssists,evenAssists,evenPoints,evenToi
0,8478178,43,D. Raddysh,D,0,0,0,1,0,1,...,2023020001,1,0.9,0,0,0,0,0,0,1060
1,8475177,44,C. de Haan,D,0,0,0,0,0,2,...,2023020001,1,0.7,0,0,0,0,0,0,649
2,8480246,48,N. Perbix,D,0,0,0,-1,2,0,...,2023020001,1,0.6,0,0,0,0,0,0,702
3,8475167,77,V. Hedman,D,0,1,1,-1,0,0,...,2023020001,1,1.9,0,0,0,0,1,1,981
4,8478416,81,E. Cernak,D,0,0,0,0,0,6,...,2023020001,1,1.3,0,0,0,0,0,0,884
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13417,8477425,28,M. Wood,L,0,0,0,-1,0,0,...,2023020373,1,0.2,0,0,0,0,0,0,827
13418,8477492,29,N. MacKinnon,C,0,1,1,-1,0,1,...,2023020373,1,1.8,0,0,0,0,1,1,1086
13419,8475193,90,T. Tatar,L,0,0,0,0,0,1,...,2023020373,1,0.2,0,0,0,0,0,0,574
13420,8481641,94,J. Kiviranta,L,0,0,0,-1,0,2,...,2023020373,1,1.8,0,0,0,0,0,0,708


In [56]:
def opposition_stats(df):
    
    opponent_stats = df.groupby('opponent').agg({
        'toi': 'sum',
        'assists': 'sum',
        'goals': 'sum',
        'shots': 'sum',
        'hits': 'sum',
        'blockedShots': 'sum',
    #     'fantasyPoints': ['sum', ('mean', lambda x: round(x.mean(), 2))],
        'fantasyPoints': 'sum',
        'gameDate': 'nunique',
        'powerPlayPoints': 'sum'
        # Add more columns as needed
    })

    opponent_stats['FPAPG'] = round(opponent_stats['fantasyPoints'] / opponent_stats['gameDate'], 2)
    opponent_stats['FPAPG_Rank'] = opponent_stats['FPAPG'].rank(ascending=False, method='min').astype(int)

    # opponent_stats['BPG'] = round(opponent_stats['blocked'] / opponent_stats['date'], 2)
    # opponent_stats['BPG_Rank'] = opponent_stats['BPG'].rank(ascending=True, method='min').astype(int)

    opponent_stats = opponent_stats.rename_axis('abbreviation')

    opponent_stats = pd.merge(opponent_stats, team_names_only, on='abbreviation')

    # opponent_stats['powerPlayPoints'] = opponent_stats['powerPlayGoals'] + opponent_stats['powerPlayAssists']

    # opponent_stats = opponent_stats.drop(columns=['powerPlayGoals', 'powerPlayAssists'])

    return opponent_stats

In [57]:
opp_all = opposition_stats(all_df)
opp_forward = opposition_stats(all_df_forwards)
opp_defense = opposition_stats(all_df_defense)
opp_centers = opposition_stats(all_df_centers)
opp_wingers = opposition_stats(all_df_wingers)

opp_even = opposition_stats(all_df_even)

opp_centers

,abbreviation,toi,assists,goals,shots,hits,blockedShots,fantasyPoints,gameDate,powerPlayPoints,FPAPG,FPAPG_Rank,name
0,ANA,132273,51,27,254,91,69,185.0,24,22,7.71,19,Anaheim Ducks
1,ARI,123848,35,25,263,128,58,160.6,23,14,6.98,30,Arizona Coyotes
2,BOS,139976,36,26,282,162,85,179.4,24,8,7.48,23,Boston Bruins
3,BUF,125005,49,31,254,141,64,189.0,25,12,7.56,21,Buffalo Sabres
4,CAR,115219,48,28,186,82,119,201.3,23,18,8.75,7,Carolina Hurricanes
5,CBJ,137255,49,34,292,82,84,200.9,26,6,7.73,18,Columbus Blue Jackets
6,CGY,132494,46,27,247,104,77,182.6,24,15,7.61,20,Calgary Flames
7,CHI,126965,51,41,286,145,66,222.1,23,22,9.66,2,Chicago Blackhawks
8,COL,129508,42,32,234,91,78,187.5,24,17,7.81,17,Colorado Avalanche
9,DAL,129986,37,26,258,135,86,174.8,22,6,7.95,14,Dallas Stars


In [58]:
# g_last_ten_df_team = pd.DataFrame()

# for index, row in team_names_only.iterrows():
#     temp_all = allG_df.loc[allG_df['team'] == row['abbreviation']].sort_values('gameDate', ascending=False)
#     five_dates = temp_all['gameDate'].unique()[:10]
#     temp_filtered = temp_all[temp_all['gameDate'].isin(five_dates)]
#     g_last_ten_df_team = pd.concat([g_last_ten_df_team, temp_filtered])
    
# g_last_ten_df_team

# gopponent_stats_10 = g_last_ten_df_team.groupby('opponent').agg({
#     'toi': 'sum',
#     'goalsAgainst': 'sum',
#     'shots': 'sum',
#     'fantasyPoints': 'sum',
#     'gameDate': 'nunique',
#     'evSaves': 'sum',
#     'evShots': 'sum',
#     'shutout': 'sum',
#     'ppSaves': 'sum',
#     'ppShots': 'sum'
# })

# gopponent_stats_10['FPAPG'] = round(gopponent_stats_10['fantasyPoints'] / gopponent_stats_10['gameDate'], 2)
# gopponent_stats_10['FPAPG_Rank'] = gopponent_stats_10['FPAPG'].rank(ascending=False, method='min').astype(int)

# gopponent_stats_10 = gopponent_stats_10.rename_axis('abbreviation')

# gopponent_stats_10 = pd.merge(gopponent_stats_10, team_names_only, on='abbreviation')

# gopponent_stats_10

In [59]:
gopponent_stats = allG_df.groupby('opponent').agg({
    'toi': 'sum',
    'goalsAgainst': 'sum',
    'shots': 'sum',
#     'fantasyPoints': ['sum', ('mean', lambda x: round(x.mean(), 2))],
    'fantasyPoints': 'sum',
    'gameDate': 'nunique',
    'evSaves': 'sum',
    'evShots': 'sum',
    'shutout': 'sum',
    'ppSaves': 'sum',
    'ppShots': 'sum'
    # Add more columns as needed
})

gopponent_stats['FPAPG'] = round(gopponent_stats['fantasyPoints'] / gopponent_stats['gameDate'], 2)
gopponent_stats['FPAPG_Rank'] = gopponent_stats['FPAPG'].rank(ascending=False, method='min').astype(int)

# gopponent_stats['SPG'] = round(gopponent_stats['shots'] / gopponent_stats['date'], 2)
# gopponent_stats['SPG_Rank'] = gopponent_stats['SPG'].rank(ascending=False, method='min').astype(int)

gopponent_stats = gopponent_stats.rename_axis('abbreviation')

gopponent_stats = pd.merge(gopponent_stats, team_names_only, on='abbreviation')

# gopponent_stats['powerPlayPoints'] = gopponent_stats['powerPlayGoals'] + gopponent_stats['powerPlayAssists']

# gopponent_stats = gopponent_stats.drop(columns=['powerPlayGoals', 'powerPlayAssists', 'id'])

gopponent_stats

,abbreviation,toi,goalsAgainst,shots,fantasyPoints,gameDate,evSaves,evShots,shutout,ppSaves,ppShots,FPAPG,FPAPG_Rank,name
0,ANA,86560,63,704,62.2,24,520,564,0,95,112,2.59,10,Anaheim Ducks
1,ARI,82966,71,622,18.2,23,452,502,1,89,109,0.79,26,Arizona Coyotes
2,BOS,85606,76,759,15.6,24,541,599,0,115,133,0.65,27,Boston Bruins
3,BUF,89679,66,721,63.0,25,542,597,1,100,110,2.52,11,Buffalo Sabres
4,CAR,83283,77,783,30.2,23,591,646,1,91,108,1.31,23,Carolina Hurricanes
5,CBJ,93456,71,776,78.0,26,574,635,2,114,124,3.00,5,Columbus Blue Jackets
6,CGY,87147,65,759,70.8,24,582,634,1,95,105,2.95,6,Calgary Flames
7,CHI,82164,53,631,81.6,23,479,524,2,75,83,3.55,2,Chicago Blackhawks
8,COL,86289,78,757,25.8,24,536,590,3,124,143,1.08,25,Colorado Avalanche
9,DAL,79252,74,653,5.8,22,473,528,1,85,100,0.26,28,Dallas Stars


In [60]:
player_bios.loc[player_bios['name'] == 'A. Ovechkin']

,playerId,name,team,position,toi,gamesPlayed,fantasyPoints,FPP60,FPPG,missedGames,gamesRemaining,fantasyPointsRemain,fullName,birthDate,shootsCatches,height,weight,roster_percent,default_pos,espnId
160,8471214,A. Ovechkin,WSH,L,24667,21,36.1,5.27,1.72,0,61,104.92,Alex Ovechkin,1985-09-17,R,75,238,99.173804,LW,3101


In [61]:
team_names_only

last_five_df_team = pd.DataFrame()

for index, row in team_names_only.iterrows():
    temp_all = all_df.loc[all_df['team'] == row['abbreviation']].sort_values('gameDate', ascending=False)
    five_dates = temp_all['gameDate'].unique()[:5]
    temp_filtered = temp_all[temp_all['gameDate'].isin(five_dates)]
    last_five_df_team = pd.concat([last_five_df_team, temp_filtered])
    
last_five_df_team

,playerId,sweaterNumber,name,position,goals,assists,points,plusMinus,pim,hits,...,team,opponent,secondaryPosition,tertiaryPosition,gameDate,gameTime,gameId,gamesPlayed,fantasyPoints,specialTeams
13277,8476459,93,M. Zibanejad,C,1,1,2,0,2,0,...,NYR,SJS,C,F,2023-12-03 00:00:00,Sunday 06:00 PM,2023020370,1,4.1,1
13268,8482109,13,A. Lafrenière,L,0,1,1,0,0,1,...,NYR,SJS,W,F,2023-12-03 00:00:00,Sunday 06:00 PM,2023020370,1,1.7,0
13260,8482073,4,B. Schneider,D,0,0,0,1,0,0,...,NYR,SJS,D,D,2023-12-03 00:00:00,Sunday 06:00 PM,2023020370,1,1.1,0
13261,8476885,8,J. Trouba,D,0,2,2,2,0,1,...,NYR,SJS,D,D,2023-12-03 00:00:00,Sunday 06:00 PM,2023020370,1,2.8,0
13262,8479323,23,A. Fox,D,0,1,1,-1,0,0,...,NYR,SJS,D,D,2023-12-03 00:00:00,Sunday 06:00 PM,2023020370,1,2.8,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10233,8475798,64,M. Granlund,C,1,1,2,0,0,1,...,SJS,VAN,C,F,2023-11-25 00:00:00,Saturday 10:00 PM,2023020314,1,4.0,1
10234,8474884,68,M. Hoffman,C,1,1,2,1,0,1,...,SJS,VAN,C,F,2023-11-25 00:00:00,Saturday 10:00 PM,2023020314,1,3.8,1
10235,8482667,72,W. Eklund,L,0,0,0,1,0,0,...,SJS,VAN,W,F,2023-11-25 00:00:00,Saturday 10:00 PM,2023020314,1,0.1,0
10219,8476988,5,M. Benning,D,0,0,0,-1,4,3,...,SJS,VAN,D,D,2023-11-25 00:00:00,Saturday 10:00 PM,2023020314,1,2.9,0


In [62]:
def summary_statistics(df):
    temp_build = df.groupby('playerId').agg({
        'name': 'first',
        'team': 'last',
        'position': 'first',
        'toi': 'sum',
        'gamesPlayed': 'sum',
        'goals': 'sum',
        'assists': 'sum',
        'specialTeams': 'sum',
        'shots': 'sum',
        'hits': 'sum',
        'blockedShots': 'sum',
        'powerPlayToi': 'sum',
        'shorthandedToi': 'sum',
        'plusMinus': 'sum',
        'fantasyPoints': 'sum',
        'secondaryPosition': 'first',
        'tertiaryPosition': 'first'
    }).reset_index()



    temp_build['FPP60'] = (temp_build['fantasyPoints'] / temp_build['toi'] * 3600).round(2)
    temp_build['FPPG'] = (temp_build['fantasyPoints'] / temp_build['gamesPlayed']).round(2)

    temp_build = temp_build.sort_values(by='FPPG', ascending=False)
    
    return temp_build

In [63]:


top_players_by_team = summary_statistics(last_five_df_team).groupby('team').apply(lambda x: x.nlargest(5, 'fantasyPoints'))
top_players_by_team.reset_index(level='team', drop=True, inplace=True)
top_players_by_team

for index, row in top_players_by_team.iterrows():
    top_players_by_team.at[index, 'available'] = 100 - round(player_bios.loc[player_bios['playerId'] == row['playerId']]['roster_percent'].iloc[0], 2)

top_players_by_team

,playerId,name,team,position,toi,gamesPlayed,goals,assists,specialTeams,shots,...,blockedShots,powerPlayToi,shorthandedToi,plusMinus,fantasyPoints,secondaryPosition,tertiaryPosition,FPP60,FPPG,available
313,8478366,F. Vatrano,ANA,R,5252,5,1,3,2,22,...,3,756,442,-6,10.8,W,F,7.40,2.16,9.04
27,8473986,A. Killorn,ANA,L,5533,5,2,2,2,17,...,2,827,303,-1,10.0,W,F,6.51,2.00,98.33
522,8480950,I. Lyubushkin,ANA,D,4552,5,0,0,0,2,...,13,13,807,-3,8.0,D,D,6.33,1.60,97.58
93,8475462,R. Gudas,ANA,D,4302,4,1,0,0,5,...,9,0,486,1,7.9,D,D,6.61,1.98,48.09
644,8484153,L. Carlsson,ANA,C,3419,3,1,2,1,6,...,2,439,3,2,6.5,C,F,6.84,2.17,87.60
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
180,8476880,T. Wilson,WSH,R,5602,5,4,0,2,23,...,3,852,637,-3,14.3,W,F,9.19,2.86,76.47
49,8474590,J. Carlson,WSH,D,7158,5,0,3,2,6,...,9,907,810,0,9.1,D,D,4.58,1.82,2.26
91,8475343,N. Dowd,WSH,C,4466,5,1,0,0,8,...,10,0,856,1,8.1,C,F,6.53,1.62,99.72
254,8477511,A. Mantha,WSH,R,4085,5,2,1,0,10,...,3,739,0,2,7.8,W,F,6.87,1.56,99.83


In [64]:

worst_five = summary_statistics(last_five_df_team).groupby('team').apply(lambda x: x.nsmallest(10, 'fantasyPoints'))
worst_five.reset_index(level='team', drop=True, inplace=True)

for index, row in worst_five.iterrows():
    worst_five.at[index, 'available'] = 100 - round(player_bios.loc[player_bios['playerId'] == row['playerId']]['roster_percent'].iloc[0], 2)
    
worst_five = worst_five.loc[worst_five['available'] <= 40]    


for index, row in worst_five.iterrows():
    print(
    f"<p><b>{player_bios.loc[player_bios['playerId'] == row['playerId']]['fullName'].iloc[0]}, "
    f"{row['position']}, {team_names_only.loc[team_names_only['abbreviation'] == row['team']]['name'].iloc[0]} "
    f"</b>({round(row['available'], 2)}% available): {round(row['fantasyPoints'], 2)} fantasy points "
    f"({row['gamesPlayed']} GP) "
)

<p><b>Mason McTavish, C, Anaheim Ducks </b>(24.48% available): 2.9 fantasy points (5 GP) 
<p><b>Logan Cooley, C, Arizona Coyotes </b>(39.33% available): 3.7 fantasy points (5 GP) 
<p><b>Dylan Cozens, C, Buffalo Sabres </b>(28.95% available): 4.5 fantasy points (5 GP) 
<p><b>Jaccob Slavin, D, Carolina Hurricanes </b>(34.69% available): 6.4 fantasy points (5 GP) 
<p><b>Brandon Montour, D, Florida Panthers </b>(13.38% available): 5.8 fantasy points (5 GP) 
<p><b>Pierre-Luc Dubois, C, Los Angeles Kings </b>(18.87% available): 3.4 fantasy points (5 GP) 
<p><b>Adrian Kempe, R, Los Angeles Kings </b>(3.88% available): 6.5 fantasy points (5 GP) 
<p><b>Matt Boldy, L, Minnesota Wild </b>(29.89% available): 6.5 fantasy points (5 GP) 
<p><b>Timo Meier, R, New Jersey Devils </b>(7.03% available): 0.5 fantasy points (1 GP) 
<p><b>Dougie Hamilton, D, New Jersey Devils </b>(6.75% available): 2.8 fantasy points (3 GP) 
<p><b>Claude Giroux, R, Ottawa Senators </b>(33.21% available): 3.2 fantasy points (

In [65]:
best_five = summary_statistics(last_five_df_team).groupby('team').apply(lambda x: x.nlargest(5, 'fantasyPoints'))
best_five.reset_index(level='team', drop=True, inplace=True)

for index, row in best_five.iterrows():
    best_five.at[index, 'available'] = 100 - round(player_bios.loc[player_bios['playerId'] == row['playerId']]['roster_percent'].iloc[0], 2)
    
best_five = best_five.loc[best_five['available'] >= 60]    


for index, row in best_five.iterrows():
    print(
    f"<p><b>{player_bios.loc[player_bios['playerId'] == row['playerId']]['fullName'].iloc[0]}, "
    f"{row['position']}, {team_names_only.loc[team_names_only['abbreviation'] == row['team']]['name'].iloc[0]} "
    f"</b>({round(row['available'], 2)}% available): {round(row['fantasyPoints'], 2)} fantasy points "
    f"({row['gamesPlayed']} GP) {row['FPPG']} FPPG"
)

<p><b>Alex Killorn, L, Anaheim Ducks </b>(98.33% available): 10.0 fantasy points (5 GP) 2.0 FPPG
<p><b>Ilya Lyubushkin, D, Anaheim Ducks </b>(97.58% available): 8.0 fantasy points (5 GP) 1.6 FPPG
<p><b>Leo Carlsson, C, Anaheim Ducks </b>(87.6% available): 6.5 fantasy points (3 GP) 2.17 FPPG
<p><b>Michael Carcone, L, Arizona Coyotes </b>(98.81% available): 12.4 fantasy points (5 GP) 2.48 FPPG
<p><b>Nick Schmaltz, C, Arizona Coyotes </b>(72.23% available): 11.3 fantasy points (5 GP) 2.26 FPPG
<p><b>Alex Kerfoot, C, Arizona Coyotes </b>(99.68% available): 10.4 fantasy points (5 GP) 2.08 FPPG
<p><b>Pavel Zacha, C, Boston Bruins </b>(65.15% available): 10.7 fantasy points (5 GP) 2.14 FPPG
<p><b>Casey Mittelstadt, C, Buffalo Sabres </b>(87.28% available): 10.7 fantasy points (5 GP) 2.14 FPPG
<p><b>Kyle Okposo, R, Buffalo Sabres </b>(99.65% available): 9.3 fantasy points (5 GP) 1.86 FPPG
<p><b>JJ Peterka, R, Buffalo Sabres </b>(81.16% available): 9.3 fantasy points (5 GP) 1.86 FPPG
<p><b>Mich

In [66]:
team_names_only


last_five_df = pd.DataFrame()

for index, row in team_names_only.iterrows():
    temp_all = allG_df.loc[allG_df['team'] == row['abbreviation']].sort_values('gameDate', ascending=False)
    five_dates = temp_all['gameDate'].unique()[:5]
    temp_filtered = temp_all[temp_all['gameDate'].isin(five_dates)]
    last_five_df = pd.concat([last_five_df, temp_filtered])
    
last_five_df

,playerId,sweaterNumber,name,position,pim,goalsAgainst,toi,team,opponent,gameDate,...,win,saves,shots,evSaves,evShots,ppSaves,ppShots,gamesPlayed,fantasyPoints,decisionType
784,8471734,32,J. Quick,G,0,5,3600,NYR,SJS,2023-12-03 00:00:00,...,4,23,28,20,24,1,2,1,-1.4,W
772,8478048,31,I. Shesterkin,G,0,3,3600,NYR,NSH,2023-12-02 00:00:00,...,4,38,41,33,34,5,6,1,5.6,W
707,8478048,31,I. Shesterkin,G,0,2,3600,NYR,DET,2023-11-29 00:00:00,...,4,25,27,19,20,5,6,1,5.0,W
683,8478048,31,I. Shesterkin,G,0,4,3519,NYR,BUF,2023-11-27 00:00:00,...,0,34,38,31,35,3,3,1,-1.2,L
595,8471734,32,J. Quick,G,0,4,3568,NYR,BOS,2023-11-25 00:00:00,...,4,27,31,22,25,3,4,1,1.4,W
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
789,8478406,29,M. Blackwood,G,0,6,3479,SJS,NYR,2023-12-03 00:00:00,...,0,30,36,27,32,2,3,1,-6.0,L
768,8478039,36,K. Kahkonen,G,0,3,3600,SJS,NJD,2023-12-01 00:00:00,...,4,44,47,41,43,3,4,1,6.8,W
752,8478406,29,M. Blackwood,G,0,3,3600,SJS,BOS,2023-11-30 00:00:00,...,0,34,37,29,31,3,4,1,0.8,L
687,8478406,29,M. Blackwood,G,0,1,3600,SJS,WSH,2023-11-27 00:00:00,...,4,33,34,27,28,6,6,1,8.6,W


In [67]:
summary_stats_last5G = last_five_df.groupby('playerId').agg({
    'name': 'first',
    'team': 'last',
    'position': 'first',
    'toi': 'sum',
    'gamesPlayed': 'sum',
    'saves': 'sum',
    'shots': 'sum',
    'start': 'sum',
    'shutout': 'sum',
    'fantasyPoints': 'sum'
}).reset_index()

summary_stats_last5G.sort_values('fantasyPoints', ascending=False).head(25)
summary_stats_last5G

for index, row in summary_stats_last5G.iterrows():
    summary_stats_last5G.at[index, 'available'] = 100 - round(player_bios.loc[player_bios['playerId'] == row['playerId']]['roster_percent'].iloc[0], 2)

summary_stats_last5G['FPP60'] = round(summary_stats_last5G['fantasyPoints'] / summary_stats_last5G['toi'] *3600, 2)
summary_stats_last5G

,playerId,name,team,position,toi,gamesPlayed,saves,shots,start,shutout,fantasyPoints,available,FPP60
0,8470594,M. Fleury,MIN,G,3575,1,28,29,1,0,7.600000e+00,85.41,7.65
1,8471734,J. Quick,NYR,G,7168,2,50,59,2,0,6.661338e-16,57.87,0.00
2,8473575,S. Varlamov,NYI,G,7366,2,67,74,2,0,7.400000e+00,79.57,3.62
3,8474593,J. Markstrom,CGY,G,10714,3,55,65,3,0,-1.000000e+00,50.47,-0.34
4,8474596,J. Allen,MTL,G,7249,2,54,63,2,0,-6.200000e+00,95.18,-3.08
...,...,...,...,...,...,...,...,...,...,...,...,...,...
61,8481033,A. Schmid,NJD,G,12073,4,90,100,3,0,6.000000e+00,96.50,1.79
62,8481035,S. Ersson,PHI,G,7800,2,54,57,2,1,1.580000e+01,99.47,7.29
63,8481611,P. Kochetkov,CAR,G,14465,4,80,90,4,0,9.000000e+00,70.35,2.24
64,8482221,D. Levi,BUF,G,2385,1,17,20,0,0,-2.600000e+00,85.44,-3.92


In [68]:
top_players_by_team

,playerId,name,team,position,toi,gamesPlayed,goals,assists,specialTeams,shots,...,blockedShots,powerPlayToi,shorthandedToi,plusMinus,fantasyPoints,secondaryPosition,tertiaryPosition,FPP60,FPPG,available
313,8478366,F. Vatrano,ANA,R,5252,5,1,3,2,22,...,3,756,442,-6,10.8,W,F,7.40,2.16,9.04
27,8473986,A. Killorn,ANA,L,5533,5,2,2,2,17,...,2,827,303,-1,10.0,W,F,6.51,2.00,98.33
522,8480950,I. Lyubushkin,ANA,D,4552,5,0,0,0,2,...,13,13,807,-3,8.0,D,D,6.33,1.60,97.58
93,8475462,R. Gudas,ANA,D,4302,4,1,0,0,5,...,9,0,486,1,7.9,D,D,6.61,1.98,48.09
644,8484153,L. Carlsson,ANA,C,3419,3,1,2,1,6,...,2,439,3,2,6.5,C,F,6.84,2.17,87.60
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
180,8476880,T. Wilson,WSH,R,5602,5,4,0,2,23,...,3,852,637,-3,14.3,W,F,9.19,2.86,76.47
49,8474590,J. Carlson,WSH,D,7158,5,0,3,2,6,...,9,907,810,0,9.1,D,D,4.58,1.82,2.26
91,8475343,N. Dowd,WSH,C,4466,5,1,0,0,8,...,10,0,856,1,8.1,C,F,6.53,1.62,99.72
254,8477511,A. Mantha,WSH,R,4085,5,2,1,0,10,...,3,739,0,2,7.8,W,F,6.87,1.56,99.83


In [69]:
df = lookahead_sked.copy()

df.rename(columns={'homeTeam': 'focusTeam', 'gameDate': 'date'}, inplace=True)
df_temp = df[['gameID', 'date', 'awayTeam', 'focusTeam', 'gameTime', 'gameDT']]
df_temp.rename(columns={'focusTeam': 'opponent', 'date': 'date', 'awayTeam': 'focusTeam'}, inplace=True)
df.rename(columns={'awayTeam': 'opponent'}, inplace=True)
df['H/A'] = 'H'
df_temp['H/A'] = 'A'
df = pd.concat([df, df_temp])
df.drop(columns=['winningGoalie'], inplace=True)

sked_stats = df
sked_stats.reset_index(inplace=True)
sked_stats = sked_stats.sort_values('gameTime')

sked_stats = sked_stats[['gameID', 'date', 'gameTime', 'focusTeam', 'H/A', 'opponent']]
# sked_stats['opponent'] = sked_stats.groupby('focusTeam')['opponent'].sum()
sked_stats


for index, row in sked_stats.iterrows():
    d = opp_defense.loc[opp_defense['abbreviation'] == row['opponent']]['FPAPG'].iloc[0]
#     drank = D.loc[D.index == row['focus']]['FPAPG_Rank'][0]
    c = opp_centers.loc[opp_centers['abbreviation'] == row['opponent']]['FPAPG'].iloc[0]
#     crank = C.loc[C.index == row['focus']]['FPAPG_Rank'][0]
    w = opp_wingers.loc[opp_wingers['abbreviation'] == row['opponent']]['FPAPG'].iloc[0]
#     wrank = W.loc[W.index == row['focus']]['FPAPG_Rank'][0]
    g = gopponent_stats.loc[gopponent_stats['abbreviation'] == row['opponent']]['FPAPG'].iloc[0]
    
    sked_stats.at[index, 'D_FPAPG'] = d
#     sked_stats.at[index, 'D_RANK'] = drank
    sked_stats.at[index, 'C_FPAPG'] = c
#     sked_stats.at[index, 'C_RANK'] = crank
    sked_stats.at[index, 'W_FPAPG'] = w
#     sked_stats.at[index, 'W_RANK'] = wrank
    sked_stats.at[index, 'G_FPAPG'] = g
    
sked_summary = sked_stats.groupby('focusTeam').sum().sort_values('D_FPAPG', ascending=False)
sked_summary['D_rank'] = sked_summary['D_FPAPG'].rank(ascending=False, method='first').astype(int)
sked_summary['C_rank'] = sked_summary['C_FPAPG'].rank(ascending=False, method='first').astype(int)
sked_summary['W_rank'] = sked_summary['W_FPAPG'].rank(ascending=False, method='first').astype(int)
sked_summary['G_rank'] = sked_summary['G_FPAPG'].rank(ascending=False, method='first').astype(int)
sked_summary

sked_summary.sort_values('D_rank', ascending=True, inplace=True)
sked_summary

d_adds = pd.DataFrame()
for index, row in sked_summary.iterrows():
    temp = top_players_by_team.loc[(top_players_by_team['team'] == index) & (top_players_by_team['position'] == 'D')]
    temp = temp.loc[temp['available'] >= 25]
    temp['D_rank'] = row['D_rank'].astype(int)
    d_adds = pd.concat([d_adds, temp])
    if row['D_rank'] == 5:
        break

sked_summary.sort_values('C_rank', ascending=True, inplace=True)
c_adds = pd.DataFrame()
for index, row in sked_summary.iterrows():
    temp = top_players_by_team.loc[(top_players_by_team['team'] == index) & (top_players_by_team['secondaryPosition'] == 'C')]
    temp = temp.loc[temp['available'] >= 25]
    temp['C_rank'] = row['C_rank'].astype(int)
    c_adds = pd.concat([c_adds, temp])
    if row['C_rank'] == 5:
        break
        
sked_summary.sort_values('W_rank', ascending=True, inplace=True)
w_adds = pd.DataFrame()
for index, row in sked_summary.iterrows():
    temp = top_players_by_team.loc[(top_players_by_team['team'] == index) & (top_players_by_team['secondaryPosition'] == 'W')]
    temp = temp.loc[temp['available'] >= 25]
    temp['W_rank'] = row['W_rank'].astype(int)
    w_adds = pd.concat([w_adds, temp])
    if row['W_rank'] == 5:
        break
        
sked_summary.sort_values('G_rank', ascending=True, inplace=True)
g_adds = pd.DataFrame()
for index, row in sked_summary.iterrows():
    temp = summary_stats_last5G.loc[(summary_stats_last5G['team'] == index)]
    temp = temp.loc[temp['available'] >= 25]
    temp['G_rank'] = row['G_rank'].astype(int)
    g_adds = pd.concat([g_adds, temp])
    if row['G_rank'] == 5:
        break
        
adds = pd.concat([d_adds, c_adds, w_adds])
adds['merged_rank'] = adds['D_rank'].combine_first(adds['C_rank']).combine_first(adds['W_rank'])
adds = adds.drop(['D_rank', 'C_rank', 'W_rank'], axis=1)
sked_stats

for index, row in adds.iterrows(): 
#     print(f"<p><b>{player_bios.loc[player_bios['playerId'] == row['playerId']]['fullName'].iloc[0]}, {row['position']}, {team_names_only.loc[team_names_only['abbreviation'] == row['team']]['name'].iloc[0]} </b>({round(row['available'], 2)}% available): {round(row['fantasyPoints'], 2)} fantasy points ({row['FPPG']} FPPG)")
    print(
    f"<p><b>{player_bios.loc[player_bios['playerId'] == row['playerId']]['fullName'].iloc[0]}, "
    f"{row['position']}, {team_names_only.loc[team_names_only['abbreviation'] == row['team']]['name'].iloc[0]} "
    f"</b>({round(row['available'], 2)}% available): {round(row['fantasyPoints'], 2)} fantasy points "
    f"({row['FPPG']} FPPG) {sked_stats.groupby('focusTeam')['opponent'].sum()[row['team']]}"
)
    
for index, row in g_adds.iterrows(): 
#     print(f"<p><b>{player_bios.loc[player_bios['playerId'] == row['playerId']]['fullName'].iloc[0]}, {row['position']}, {team_names_only.loc[team_names_only['abbreviation'] == row['team']]['name'].iloc[0]} </b>({round(row['available'], 2)}% available): {round(row['fantasyPoints'], 2)} fantasy points ({row['FPPG']} FPPG)")
    print(
    f"<p><b>{player_bios.loc[player_bios['playerId'] == row['playerId']]['fullName'].iloc[0]}, "
    f"{row['position']}, {team_names_only.loc[team_names_only['abbreviation'] == row['team']]['name'].iloc[0]} "
    f"</b>({round(row['available'], 2)}% available): {round(row['fantasyPoints'], 2)} fantasy points "
    f"({row['FPP60']} FPP60) {sked_stats.groupby('focusTeam')['opponent'].sum()[row['team']]}"
)

<p><b>Jake Middleton, D, Minnesota Wild </b>(80.51% available): 11.5 fantasy points (2.3 FPPG) EDMSEAVANCGY
<p><b>Jonas Brodin, D, Minnesota Wild </b>(69.76% available): 9.6 fantasy points (1.92 FPPG) EDMSEAVANCGY
<p><b>Brady Skjei, D, Carolina Hurricanes </b>(44.66% available): 8.1 fantasy points (1.62 FPPG) WPGVANCGYEDM
<p><b>Justin Barron, D, Montréal Canadiens </b>(98.41% available): 9.1 fantasy points (1.82 FPPG) SEABUFNSHLAK
<p><b>Gustav Lindström, D, Montréal Canadiens </b>(99.94% available): 8.9 fantasy points (1.78 FPPG) SEABUFNSHLAK
<p><b>Kaiden Guhle, D, Montréal Canadiens </b>(60.36% available): 7.6 fantasy points (1.52 FPPG) SEABUFNSHLAK
<p><b>Yakov Trenin, C, Nashville Predators </b>(99.09% available): 8.5 fantasy points (1.7 FPPG) TORMTLTBLCHI
<p><b>Gustav Nyquist, C, Nashville Predators </b>(92.02% available): 7.8 fantasy points (1.56 FPPG) TORMTLTBLCHI
<p><b>Dawson Mercer, C, New Jersey Devils </b>(42.87% available): 12.6 fantasy points (2.52 FPPG) CGYEDMSEAVAN
<p><b>O

In [70]:
sked_summary

,gameID,D_FPAPG,C_FPAPG,W_FPAPG,G_FPAPG,D_rank,C_rank,W_rank,G_rank
focusTeam,,,,,,,,,
STL,8092081590,35.04,31.35,30.62,10.13,11,10,10,1
DAL,8092081569,34.94,31.55,30.74,9.45,12,9,9,2
NSH,8092081618,37.60,36.05,35.24,9.25,7,1,2,3
CHI,8092081621,35.90,30.38,31.70,9.24,9,14,5,4
VGK,8092081606,34.20,31.05,33.08,8.60,14,11,3,5
DET,6069061185,27.68,25.07,26.93,8.52,19,16,18,6
MIN,8092081622,40.13,34.61,30.37,7.96,2,3,12,7
NJD,8092081620,40.13,34.61,30.37,7.96,1,2,11,8
TBL,8092081580,38.08,31.98,30.84,7.74,6,7,8,9


In [71]:
lookahead_sked = lookahead_sked.sort_values('gameTime')
lookahead_sked

,gameID,gameDate,gameTime,awayTeam,homeTeam,gameDT,winningGoalie
1122,2023020405,2023-12-08,Friday 07:00 PM,STL,CBJ,2023-12-09 00:00:00,0
299,2023020406,2023-12-08,Friday 07:00 PM,PIT,FLA,2023-12-09 00:00:00,0
1159,2023020407,2023-12-08,Friday 09:00 PM,MIN,EDM,2023-12-09 02:00:00,0
709,2023020375,2023-12-04,Monday 07:00 PM,DAL,TBL,2023-12-05 00:00:00,0
627,2023020374,2023-12-04,Monday 07:00 PM,PIT,PHI,2023-12-05 00:00:00,0
464,2023020377,2023-12-04,Monday 07:30 PM,CAR,WPG,2023-12-05 00:30:00,0
915,2023020376,2023-12-04,Monday 07:30 PM,SEA,MTL,2023-12-05 00:30:00,0
544,2023020378,2023-12-04,Monday 09:00 PM,WSH,ARI,2023-12-05 02:00:00,0
94,2023020379,2023-12-04,Monday 10:00 PM,STL,VGK,2023-12-05 03:00:00,0
53,2023020408,2023-12-09,Saturday 01:00 PM,ARI,BOS,2023-12-09 18:00:00,0


In [72]:
for index, row in lookahead_sked.iterrows():
    
    date_object_utc = datetime.strptime(row['gameDT'], "%Y-%m-%d %H:%M:%S").replace(tzinfo=pytz.UTC)
    date_object_et = date_object_utc.astimezone(pytz.timezone("US/Eastern"))
    formatted_time = date_object_et.strftime("%I:%M p.m.").lstrip('0').replace("PM", "p.m.")
    
    date_string_et = date_object_et.strftime("%Y-%m-%d %H:%M:%S%z")
    date_object = datetime.strptime(date_string_et, "%Y-%m-%d %H:%M:%S%z")    
    day_of_week = date_object.strftime("%A")
    
    home_abbr = row['homeTeam']
    home_top5 = top_players_by_team.loc[top_players_by_team['team'] == home_abbr]
    home_goalies = summary_stats_last5G.loc[summary_stats_last5G['team'] == home_abbr]

    away_abbr = row['awayTeam']
    away_top5 = top_players_by_team.loc[top_players_by_team['team'] == away_abbr]
    away_goalies = summary_stats_last5G.loc[summary_stats_last5G['team'] == away_abbr]
    
    home_team = team_names_only.loc[team_names_only['abbreviation'] == home_abbr]['name'].iloc[0]
    away_team = team_names_only.loc[team_names_only['abbreviation'] == away_abbr]['name'].iloc[0]
        
    print(f"\033[1m{away_team} at {home_team}\033[0m")
    print(f"{formatted_time}")
    print()
    print(f"\033[1mForwards against {home_abbr} score\033[0m: {opp_forward.loc[opp_forward['abbreviation'] == home_abbr, 'FPAPG'].values[0]} FPAPG, Rank: {opp_forward.loc[opp_forward['abbreviation'] == home_abbr, 'FPAPG_Rank'].values[0]}")
    print(f"\033[1mDefence against {home_abbr} score\033[0m: {opp_defense.loc[opp_defense['abbreviation'] == home_abbr, 'FPAPG'].values[0]} FPAPG, Rank: {opp_defense.loc[opp_defense['abbreviation'] == home_abbr, 'FPAPG_Rank'].values[0]}")
    print(f"\033[1mTop {away_abbr} Players Fantasy Points Last 5 \033[0m")
    for index, row in away_top5.iterrows():
        print(f"<p><b>{player_bios.loc[player_bios['playerId'] == row['playerId']]['fullName'].iloc[0]}, {row['position']}, {away_team} </b>({round(row['available'], 2)}% available): {round(row['fantasyPoints'], 2)} fantasy points ({row['FPPG']} FPPG)")
    print()
    print(f"\033[1mGoalies against {home_abbr} score\033[0m: {gopponent_stats.loc[gopponent_stats['name'] == home_team, 'FPAPG'].values[0]} FPAPG, Rank: {gopponent_stats.loc[gopponent_stats['name'] == home_team, 'FPAPG_Rank'].values[0]}")
    for index, row in away_goalies.iterrows():
        print(f"{player_bios.loc[player_bios['playerId'] == row['playerId']]['fullName'].iloc[0]} ({round(row['available'], 2)}% available) last five: {row['start']} starts, FPP60: {row['FPP60']}, FP: {round(row['fantasyPoints'], 2)}")
    print()
    print(f"\033[1mForwards against {away_abbr} score\033[0m: {opp_forward.loc[opp_forward['abbreviation'] == away_abbr, 'FPAPG'].values[0]} FPAPG, Rank: {opp_forward.loc[opp_forward['abbreviation'] == away_abbr, 'FPAPG_Rank'].values[0]}")
    print(f"\033[1mDefence against {away_abbr} score\033[0m: {opp_defense.loc[opp_defense['abbreviation'] == away_abbr, 'FPAPG'].values[0]} FPAPG, Rank: {opp_defense.loc[opp_defense['abbreviation'] == away_abbr, 'FPAPG_Rank'].values[0]}")
    print(f"\033[1mTop {home_abbr} Players Fantasy Points Last 5 \033[0m")
    for index, row in home_top5.iterrows():
        print(f"<p><b>{player_bios.loc[player_bios['playerId'] == row['playerId']]['fullName'].iloc[0]}, {row['position']}, {home_team}</b>({round(row['available'], 2)}% available): {round(row['fantasyPoints'], 2)} fantasy points ({row['FPPG']} FPPG)")

    print()
    print(f"\033[1mGoalies against {away_abbr} score\033[0m: {gopponent_stats.loc[gopponent_stats['name'] == away_team, 'FPAPG'].values[0]} FPAPG, Rank: {gopponent_stats.loc[gopponent_stats['name'] == away_team, 'FPAPG_Rank'].values[0]}")
    for index, row in home_goalies.iterrows():
        print(f"{player_bios.loc[player_bios['playerId'] == row['playerId']]['fullName'].iloc[0]} ({round(row['available'], 2)}% available) last five: {row['start']} starts, FPP60: {row['FPP60']}, FP: {round(row['fantasyPoints'], 2)}")
    print()
    print('-'*25)
    print()

St. Louis Blues at Columbus Blue Jackets
7:00 p.m.

Forwards against CBJ score: 16.1 FPAPG, Rank: 15
Defence against CBJ score: 9.9 FPAPG, Rank: 7
Top STL Players Fantasy Points Last 5 
<p><b>Jake Neighbours, L, St. Louis Blues </b>(91.74% available): 16.1 fantasy points (3.22 FPPG)
<p><b>Pavel Buchnevich, L, St. Louis Blues </b>(25.14% available): 12.2 fantasy points (2.44 FPPG)
<p><b>Torey Krug, D, St. Louis Blues </b>(90.98% available): 10.4 fantasy points (2.08 FPPG)
<p><b>Robert Thomas, C, St. Louis Blues </b>(24.36% available): 9.4 fantasy points (1.88 FPPG)
<p><b>Brayden Schenn, C, St. Louis Blues </b>(86.42% available): 8.1 fantasy points (1.62 FPPG)

Goalies against CBJ score: 3.0 FPAPG, Rank: 5
Jordan Binnington (52.05% available) last five: 3 starts, FPP60: 1.9, FP: 6.8
Joel Hofer (98.17% available) last five: 2 starts, FPP60: -6.15, FP: -8.2

Forwards against STL score: 15.3 FPAPG, Rank: 22
Defence against STL score: 7.67 FPAPG, Rank: 32
Top CBJ Players Fantasy Points Last 

<p><b>Roman Josi, D, Nashville Predators</b>(0.18% available): 14.3 fantasy points (2.86 FPPG)
<p><b>Filip Forsberg, L, Nashville Predators</b>(8.86% available): 10.3 fantasy points (2.06 FPPG)
<p><b>Yakov Trenin, C, Nashville Predators</b>(99.09% available): 8.5 fantasy points (1.7 FPPG)
<p><b>Michael McCarron, R, Nashville Predators</b>(99.94% available): 8.4 fantasy points (1.68 FPPG)
<p><b>Gustav Nyquist, C, Nashville Predators</b>(92.02% available): 7.8 fantasy points (1.56 FPPG)

Goalies against TBL score: 1.81 FPAPG, Rank: 17
Juuse Saros (11.31% available) last five: 4 starts, FPP60: 4.55, FP: 15.2
Kevin Lankinen (99.33% available) last five: 1 starts, FPP60: -2.87, FP: -4.6

-------------------------

Dallas Stars at Washington Capitals
8:00 p.m.

Forwards against WSH score: 14.87 FPAPG, Rank: 24
Defence against WSH score: 8.41 FPAPG, Rank: 27
Top DAL Players Fantasy Points Last 5 
<p><b>Joe Pavelski, C, Dallas Stars </b>(10.46% available): 14.4 fantasy points (2.88 FPPG)
<p><b

In [73]:
temp_team_df = pd.DataFrame()

for index, row in team_names_only.iterrows():
    temp_all = all_df.loc[all_df['team'] == row['abbreviation']].sort_values('gameDate', ascending=False)
    five_dates = temp_all['gameDate'].unique()[:6]
    temp_filtered = temp_all[temp_all['gameDate'].isin(five_dates)]
    temp_team_df = pd.concat([temp_team_df, temp_filtered])
    
temp_team_df = temp_team_df.loc[(temp_team_df['team'] == 'STL') & (temp_team_df['position'] == 'D')]
temp_team_df = summary_statistics(temp_team_df)
# temp_team_df['powerPlayToi'] = f"{(temp_team_df['powerPlayToi'] // 60)}:{(temp_team_df['powerPlayToi'] % 60)}"
temp_team_df

,playerId,name,team,position,toi,gamesPlayed,goals,assists,specialTeams,shots,hits,blockedShots,powerPlayToi,shorthandedToi,plusMinus,fantasyPoints,secondaryPosition,tertiaryPosition,FPP60,FPPG
3,8476792,T. Krug,STL,D,7807,6,0,4,3,16,6,7,1034,195,-8,11.2,D,D,5.16,1.87
4,8476892,C. Parayko,STL,D,8372,6,2,0,0,16,6,9,52,1073,-5,10.7,D,D,4.60,1.78
1,8475181,N. Leddy,STL,D,7970,6,1,2,1,3,7,8,8,1075,-1,9.5,D,D,4.29,1.58
2,8475753,J. Faulk,STL,D,7680,6,0,2,1,12,7,9,591,584,-7,8.9,D,D,4.17,1.48
5,8481006,T. Tucker,STL,D,1691,2,0,0,0,1,4,4,0,0,3,2.5,D,D,5.32,1.25
0,8474618,M. Scandella,STL,D,4581,6,0,1,0,3,5,5,6,391,7,4.3,D,D,3.38,0.72
6,8481059,S. Perunovich,STL,D,3448,4,0,1,0,6,2,2,632,0,1,2.8,D,D,2.92,0.70


In [74]:
temp_player = all_df.loc[all_df['name'] == 'A. Svechnikov']
temp_player = temp_player.sort_values('gameDate', ascending=True).head(10)
summary_statistics(temp_player)

,playerId,name,team,position,toi,gamesPlayed,goals,assists,specialTeams,shots,hits,blockedShots,powerPlayToi,shorthandedToi,plusMinus,fantasyPoints,secondaryPosition,tertiaryPosition,FPP60,FPPG
0,8480830,A. Svechnikov,CAR,R,9928,10,0,6,1,21,23,3,1085,4,0,12.4,W,F,4.5,1.24


In [75]:
player_bios.sort_values('fantasyPoints', ascending=False, inplace=True)
player_bios['rank'] = player_bios['fantasyPoints'].rank(ascending=False, method='first')
player_bios.loc[player_bios['team'] == 'NYR']

,playerId,name,team,position,toi,gamesPlayed,fantasyPoints,FPP60,FPPG,missedGames,...,fantasyPointsRemain,fullName,birthDate,shootsCatches,height,weight,roster_percent,default_pos,espnId,rank
10,8478550,A. Panarin,NYR,L,26852,23,67.5,9.05,2.93,0,...,172.87,Artemi Panarin,1991-10-30,R,72,175,98.120838,LW,3891952,13.0
14,8476885,J. Trouba,NYR,D,31471,23,63.5,7.26,2.76,0,...,162.84,Jacob Trouba,1994-02-26,R,75,209,97.629640,D,2976839,18.0
59,8475184,C. Kreider,NYR,L,25847,23,52.9,7.37,2.30,0,...,135.70,Chris Kreider,1991-04-30,L,75,233,89.237791,LW,5833,53.0
80,8476389,V. Trocheck,NYR,C,27805,23,49.8,6.45,2.17,0,...,128.03,Vincent Trocheck,1993-07-11,R,71,187,82.261349,C,2563036,66.0
95,8476459,M. Zibanejad,NYR,C,27152,23,46.9,6.22,2.04,0,...,120.36,Mika Zibanejad,1993-04-18,R,74,201,97.167536,C,2562637,88.0
107,8478048,I. Shesterkin,NYR,G,48884,14,46.2,3.40,3.30,0,...,117.83,Igor Shesterkin,1995-12-30,L,73,197,99.632603,G,3151297,96.0
174,8471734,J. Quick,NYR,G,30796,9,39.6,4.63,4.40,0,...,101.00,Jonathan Quick,1986-01-21,L,73,215,42.127112,G,3634,162.0
189,8476979,E. Gustafsson,NYR,D,25736,23,38.5,5.39,1.67,0,...,98.53,Erik Gustafsson,1992-03-14,L,73,189,23.824931,D,3891922,172.0
218,8480817,K. Miller,NYR,D,31101,23,36.5,4.22,1.59,0,...,93.81,K'Andre Miller,2000-01-21,L,77,210,63.686263,D,4352770,195.0
24,8479323,A. Fox,NYR,D,16414,13,33.9,7.44,2.61,0,...,153.99,Adam Fox,1998-02-17,R,71,185,99.602773,D,4197146,225.0


In [76]:
last_five_df_team
hurricanes = last_five_df_team.loc[last_five_df_team['team'] == 'CAR'].sort_values('gameDate')
summary_statistics(hurricanes)

,playerId,name,team,position,toi,gamesPlayed,goals,assists,specialTeams,shots,hits,blockedShots,powerPlayToi,shorthandedToi,plusMinus,fantasyPoints,secondaryPosition,tertiaryPosition,FPP60,FPPG
12,8478427,S. Aho,CAR,C,5606,5,3,3,1,21,2,1,763,467,4,12.3,C,F,7.90,2.46
10,8477962,B. Lemieux,CAR,L,573,1,1,0,0,3,0,0,0,0,1,2.3,W,F,14.45,2.30
16,8480830,A. Svechnikov,CAR,R,3694,4,1,4,3,12,5,0,578,40,1,9.2,W,F,8.97,2.30
6,8476882,T. Teravainen,CAR,L,5076,5,1,3,2,13,0,3,700,347,2,8.8,W,F,6.24,1.76
11,8478047,M. Bunting,CAR,L,4179,5,2,3,1,8,0,0,396,0,5,8.3,W,F,7.15,1.66
5,8476869,B. Skjei,CAR,D,6415,5,2,0,1,10,1,5,27,730,0,8.1,D,D,4.55,1.62
0,8470613,B. Burns,CAR,D,6283,5,0,3,2,14,2,4,713,527,2,7.6,D,D,4.35,1.52
4,8476474,S. Noesen,CAR,R,3596,5,2,1,0,10,6,1,410,3,4,7.1,W,F,7.11,1.42
14,8480039,M. Necas,CAR,C,4852,5,0,4,0,12,3,3,522,4,1,7.0,C,F,5.19,1.40
18,8482093,S. Jarvis,CAR,C,5583,5,1,3,0,5,3,2,745,728,5,6.8,C,F,4.38,1.36


In [77]:
top_players_by_team.sort_values('fantasyPoints', ascending=False).head(60)

,playerId,name,team,position,toi,gamesPlayed,goals,assists,specialTeams,shots,...,blockedShots,powerPlayToi,shorthandedToi,plusMinus,fantasyPoints,secondaryPosition,tertiaryPosition,FPP60,FPPG,available
319,8478402,C. McDavid,EDM,C,6123,5,2,12,6,10,...,5,888,45,1,22.9,C,F,13.46,4.58,0.12
146,8476453,N. Kucherov,TBL,R,6538,5,2,9,6,28,...,1,1539,0,-1,19.3,W,F,10.63,3.86,0.13
428,8479675,T. Moore,LAK,L,5159,5,5,2,2,17,...,5,576,421,5,17.7,W,F,12.35,3.54,45.64
511,8480865,N. Dobson,NYI,D,8271,5,0,6,2,9,...,18,817,449,3,17.4,D,D,7.57,3.48,4.40
357,8478550,A. Panarin,NYR,L,5797,5,5,4,3,17,...,0,757,4,1,17.2,W,F,10.68,3.44,1.88
560,8481559,J. Hughes,NJD,C,6946,5,3,5,2,38,...,2,1169,17,3,17.0,C,F,8.81,3.40,0.29
238,8477492,N. MacKinnon,COL,C,7120,5,2,5,3,19,...,8,1451,7,2,16.7,C,F,8.44,3.34,0.10
592,8482089,J. Neighbours,STL,L,5519,5,5,0,1,17,...,5,897,282,-2,16.1,W,F,10.50,3.22,91.74
114,8475786,Z. Hyman,EDM,L,4605,4,4,4,2,23,...,1,643,12,4,16.1,W,F,12.59,4.03,7.72
265,8477934,L. Draisaitl,EDM,C,6048,5,4,3,5,12,...,2,898,274,-1,16.0,C,F,9.52,3.20,0.10


In [78]:
preNich = all_df.loc[(all_df['name'] == 'M. Hoffman') & (all_df['gameDate'] <= '2023-11-12')]
postNich = all_df.loc[(all_df['name'] == 'M. Hoffman') & (all_df['gameDate'] > '2023-11-12')]

preNichSum = summary_statistics(preNich)
postNichSum = summary_statistics(postNich)

postNichSum

,playerId,name,team,position,toi,gamesPlayed,goals,assists,specialTeams,shots,hits,blockedShots,powerPlayToi,shorthandedToi,plusMinus,fantasyPoints,secondaryPosition,tertiaryPosition,FPP60,FPPG
0,8474884,M. Hoffman,SJS,C,9174,10,6,2,2,17,4,5,937,0,1,19.6,C,F,7.69,1.96


In [79]:
summary_statistics(all_df).sort_values('FPP60', ascending=False).head(60)

,playerId,name,team,position,toi,gamesPlayed,goals,assists,specialTeams,shots,hits,blockedShots,powerPlayToi,shorthandedToi,plusMinus,fantasyPoints,secondaryPosition,tertiaryPosition,FPP60,FPPG
637,8481559,J. Hughes,NJD,C,21649,17,9,21,14,82,4,6,3676,64,0,57.6,C,F,9.58,3.39
156,8476453,N. Kucherov,TBL,R,31088,24,15,25,20,117,13,9,5895,10,-3,82.5,W,F,9.55,3.44
235,8477380,J. Brodzinski,NYR,C,2451,4,0,4,0,9,0,3,184,0,2,6.4,C,F,9.40,1.60
309,8477956,D. Pastrnak,BOS,R,28095,24,14,22,15,114,24,4,5553,119,8,73.3,W,F,9.39,3.05
4,8470794,J. Pavelski,DAL,C,21316,22,11,13,7,54,20,19,3439,71,1,55.4,C,F,9.36,2.52
167,8476468,J. Miller,VAN,C,30022,25,13,23,17,56,52,19,6256,2712,11,77.8,C,F,9.33,3.11
594,8480990,D. Chisholm,WPG,D,778,1,0,1,0,0,0,2,113,0,0,2.0,D,D,9.25,2.00
395,8478550,A. Panarin,NYR,L,26852,23,15,20,14,89,1,3,4426,4,3,67.5,W,F,9.05,2.93
420,8479318,A. Matthews,TOR,C,28457,22,16,9,8,93,31,28,4624,1593,0,71.4,C,F,9.03,3.25
306,8477952,R. Fabbri,DET,C,8852,11,7,2,3,18,4,5,1019,52,-1,22.2,C,F,9.03,2.02
